#### <b>리뷰 태그 데이터 세트 분석</b>

In [ ]:
# 압축 파일 해제
!unzip w2v.zip

Archive:  w2v.zip
  inflating: cluster_data.pickle     
  inflating: idx2word.pickle         
  inflating: sql_to_csv.py           
  inflating: train_data.pickle       
  inflating: word2idx.pickle         
  inflating: x.pickle                
  inflating: y.pickle                


In [ ]:
from itertools import permutations
import pickle

# 단어(word)를 인덱스(index)로 매핑한 사전 자료형
with open('./word2idx.pickle', 'rb') as f:
    word2idx = pickle.load(f)
print("총 단어의 수:", len(word2idx))

# 인덱스(index)를 단어(word)로 매핑한 사전 자료형
with open('./idx2word.pickle', 'rb') as f:
    idx2word = pickle.load(f)
print("총 인덱스의 수:", len(word2idx))

# 개별 리뷰에 담긴 태그에 대한 정보
# (전체 리뷰 개수 X 해당 리뷰의 태그 인덱스)의 크기를 가지는 2차원 리스트
with open('./cluster_data.pickle', 'rb') as f:
    cluster_data = pickle.load(f)

print(cluster_data[:5])

총 단어의 수: 267
총 인덱스의 수: 267
[[1, 2, 3], [4, 5, 1, 6, 7], [8, 7, 2], [3, 9, 2, 10], [7]]


* 현재 데이터 상으로 인덱스가 1부터 시작하므로, 총 데이터 크기를 268로 설정한다.

In [ ]:
for i in range(5):
    review = cluster_data[i]
    print(f"[리뷰 {i + 1}] [", end='')
    for x in review[:-1]:
        print(idx2word[x], end=', ')
    print(idx2word[review[-1]] + "]")

[리뷰 1] [맞춤 케어를 잘해줘요, 친절해요, 매장이 청결해요]
[리뷰 2] [주차하기 편해요, 좋은 제품을 사용해요, 맞춤 케어를 잘해줘요, 반려동물을 잘 다뤄줘요, 시술이 꼼꼼해요]
[리뷰 3] [원하는 스타일로 잘해줘요, 시술이 꼼꼼해요, 친절해요]
[리뷰 4] [매장이 청결해요, 상담이 자세해요, 친절해요, 손상이 적어요]
[리뷰 5] [시술이 꼼꼼해요]


In [ ]:
# 각 리뷰마다 2개의 태그가 서로를 정답으로 삼도록 학습 데이터 구성
original_train_dataset = []
for review in cluster_data:
    for tag in permutations(review, 2):
        original_train_dataset.append(tag)

#### <b>라이브러리 불러오기</b>

* 필요한 라이브러리를 불러온다.

In [ ]:
import numpy as np # 데이터 처리를 위한 NumPy 라이브러리 불러오기
import matplotlib.pyplot as plt # 데이터 시각화 라이브러리 불러오기

import torch # PyTorch 라이브러리 불러오기

#### <b>데이터 세트 및 모델 정의</b>

In [ ]:
import copy
from torch.utils.data.dataset import Dataset


class CustomDataset(Dataset):
    # ① __init__(): 불러올 데이터에 대한 개괄적인 내용
    def __init__(self):
        # 전체 데이터를 담는 리스트
        self.dataset = copy.deepcopy(original_train_dataset)

    # ② __len__(): 현재 데이터 세트에 포함된 총 데이터의 개수
    def __len__(self):
      return len(self.dataset)

    # ③ __getitem__(): 특정한 인덱스에 해당하는 데이터를 반환하는 함수
    def __getitem__(self, idx):
        x = self.dataset[idx][0]
        y = self.dataset[idx][1]

        # 입력 데이터(x)와 출력 데이터(y)를 반환
        return x, y

In [ ]:
train_dataset = CustomDataset()

In [ ]:
print(train_dataset[0][0], train_dataset[0][1])

1 2


In [ ]:
import torch.nn as nn # PyTorch Neural Networks (nn) 라이브러리


# 태그(tag)에서 특징을 추출하는 인코더 네트워크
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()

        # 267차원 원-핫 인코딩을 2차원의 임베딩으로 변환
        # 원-핫 인코딩: [0, 0, 0, 1, 0, 0, 0, ...] → 차원: 268
        self.layer = nn.Embedding(268, 2)

    # 특징 추출 수행
    def forward(self, x):
        x = self.layer(x)

        return x

In [ ]:
Encoder()

Encoder(
  (layer): Embedding(268, 2)
)

In [ ]:
# 특징에서 다시 원본 태그 정보를 복원하는 디코더 네트워크
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()

        # 다시 2차원의 임베딩을 268차원의 공간으로 매핑
        self.layer = nn.Linear(2, 268)

    # 특징에서 다시 태그(tag) 복원 수행
    def forward(self, x):
        x = self.layer(x)

        return x

In [ ]:
Decoder()

Decoder(
  (layer): Linear(in_features=2, out_features=268, bias=True)
)

In [ ]:
# 전체 Word to Vector (Word2Vec) 모델 정의
class Word2Vec(nn.Module):
    def __init__(self):
        super(Word2Vec, self).__init__()

        self.encoder = Encoder() # 인코더 네트워크 초기화
        self.decoder = Decoder() # 디코더 네트워크 초기화

    # 하나의 입력이 주어지면, 인코더와 디코더를 거친 결과를 출력
    def forward(self, x):
        x = self.encoder(x) # 인코더를 이용해 특징 추출
        x = self.decoder(x) # 디코더를 이용해 태그 정보 복원

        return x

In [ ]:
Word2Vec()

Word2Vec(
  (encoder): Encoder(
    (layer): Embedding(268, 2)
  )
  (decoder): Decoder(
    (layer): Linear(in_features=2, out_features=268, bias=True)
  )
)

#### <b>딥러닝 모델 학습하기</b>

In [ ]:
import tqdm # 학습 과정 시각화 라이브러리

from torch.utils.data.dataloader import DataLoader
from torch.optim.adam import Adam

train_dataset = CustomDataset() # 학습을 위한 데이터 세트
train_loader = DataLoader(train_dataset, batch_size=1024)

In [ ]:
print(original_train_dataset[0:20])

[(1, 2), (1, 3), (2, 1), (2, 3), (3, 1), (3, 2), (4, 5), (4, 1), (4, 6), (4, 7), (5, 4), (5, 1), (5, 6), (5, 7), (1, 4), (1, 5), (1, 6), (1, 7), (6, 4), (6, 5)]


In [ ]:
model = Word2Vec().cuda() # 학습할 전체 모델을 GPU에 업로드

# 하이퍼 파라미터 설정
lr = 0.001
optim = Adam(params=model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

for epoch in range(30):
    iterator = tqdm.tqdm(train_loader) # 학습 과정을 시각화하기 위해 tqdm 사용

    for inputs, targets in iterator:
        # 입력과 출력을 GPU에 업로드
        inputs = inputs.cuda()
        targets = torch.nn.functional.one_hot(targets, num_classes=268)
        targets = targets.type(torch.FloatTensor).cuda()
        pred = model(inputs) # ② 모델에 입력하여 추론 결과 확인
        pred = nn.Softmax(dim=0)(pred)

        loss = criterion(pred, targets) # ③ 모델의 예측 결과와 정답을 비교하여 손실 계산
        loss.backward() # ④ 역전파를 통해 모델의 가중치에 대한 기울기 계산
        optim.step() # ⑤ 계산된 기울기를 이용해 모델 가중치 업데이트

        # 로그 시각화
        iterator.set_description(f"[Epoch {epoch + 1}] loss: {loss.item()}")

[Epoch 30] loss: 5.583938121795654: 100%|██████████| 502/502 [00:04<00:00, 102.77it/s]


In [ ]:
# 학습된 모델 저장
torch.save(model.state_dict(), "./AE.pt") # 학습된 모델의 가중치를 사전 자료형으로 저장

In [ ]:
# 학습된 모델 성능 평가
model = Word2Vec()
model.load_state_dict(torch.load("./AE.pt"))
model = model.cuda()

In [ ]:
input = torch.Tensor([i]).type(torch.LongTensor).cuda()
print(input.shape)

torch.Size([1])


In [ ]:
features = []

with torch.no_grad():
    for i in range(1, 268):
        input = torch.Tensor([i]).type(torch.LongTensor).cuda()
        feature = model.encoder(input)
        features.append(feature[0].cpu().numpy())

for i in range(1, 268):
    print(i, idx2word[i], features[i - 1])

1 맞춤 케어를 잘해줘요 [16.067236 -9.417645]
2 친절해요 [  0.8804438 -11.93785  ]
3 매장이 청결해요 [-8.1773405 10.370701 ]
4 주차하기 편해요 [ 6.5117893 -1.60577  ]
5 좋은 제품을 사용해요 [-12.990045  14.324648]
6 반려동물을 잘 다뤄줘요 [ 5.7493625 -7.3975906]
7 시술이 꼼꼼해요 [-13.147782  14.677221]
8 원하는 스타일로 잘해줘요 [ 16.991589 -11.436204]
9 상담이 자세해요 [16.69204  -9.554464]
10 손상이 적어요 [  1.7982157 -13.25756  ]
11 트렌디해요 [-3.1967888 10.150814 ]
12 음식이 맛있어요 [17.330664   6.3095326]
13 재료가 신선해요 [-16.6757     -6.1807575]
14 건강한 맛이에요 [ -5.4414515 -16.751434 ]
15 뷰가 좋아요 [ 7.279092 15.567213]
16 야외 공간이 멋져요 [0.21474168 1.8303906 ]
17 특별한 메뉴가 있어요 [-15.098286  -6.602959]
18 양이 많아요 [-16.745214 -11.275472]
19 가성비가 좋아요 [-12.284507   -4.7531223]
20 매장이 넓어요 [-10.730827 -16.60344 ]
21 단체모임 하기 좋아요 [-1.387674 16.847145]
22 혼밥하기 좋아요 [-4.199916 -2.930002]
23 현지 맛에 가까워요 [-5.689586  -1.2301569]
24 화장실이 깨끗해요 [-12.575831 -16.236105]
25 인테리어가 멋져요 [ -7.05695  -19.325089]
26 특별한 날 가기 좋아요 [-4.153242 -1.840929]
27 스타일 추천을 잘해줘요 [-2.039183 13.692041]
28 관리법을 잘 알려줘요 [-6.

#### <b>가까운 단어 추출하기</b>

In [ ]:
import numpy as np

target = 1
input = torch.Tensor([target]).type(torch.LongTensor).cuda()
target_feature = model.encoder(input)
target_feature = target_feature[0].detach().cpu().numpy()

print(target, idx2word[target])

close_features = []
for i in range(1, 268):
    feature = features[i - 1]
    distance = np.linalg.norm(np.array(target_feature) - np.array(feature))
    close_features.append([distance, i, idx2word[i]])
close_features.sort()
print(close_features[:5])

1 맞춤 케어를 잘해줘요
[[0.0, 1, '맞춤 케어를 잘해줘요'], [0.6238771, 142, '위생적이에요'], [0.6396084, 9, '상담이 자세해요'], [0.937787, 78, '작업이 신속해요'], [1.3580788, 74, '서비스가 좋아요']]


In [ ]:
import numpy as np

target = 30
input = torch.Tensor([target]).type(torch.LongTensor).cuda()
target_feature = model.encoder(input)
target_feature = target_feature[0].detach().cpu().numpy()

print(target, idx2word[target])

close_features = []
for i in range(1, 268):
    feature = features[i - 1]
    distance = np.linalg.norm(np.array(target_feature) - np.array(feature))
    close_features.append([distance, i, idx2word[i]])
close_features.sort()
print(close_features[:5])

30 손이 빨라요
[[0.0, 30, '손이 빨라요'], [1.8063122, 182, '위생적으로 케어해줘요'], [1.9547025, 114, '휴게시설이 잘 되어있어요'], [2.423202, 251, '마사지가 시원해요'], [3.3075578, 14, '건강한 맛이에요']]
